# CART算法实现

In [15]:
import numpy as np
import math
from sklearn.model_selection import train_test_split

In [16]:
# 创建测试数据集
def createDataSet():
    dataSet = [[0, 0, 0, 0, 'no'],  # 数据集
               [0, 0, 0, 1, 'no'],
               [0, 1, 0, 1, 'yes'],
               [0, 1, 1, 0, 'yes'],
               [0, 0, 0, 0, 'no'],
               [1, 0, 0, 0, 'no'],
               [1, 0, 0, 1, 'no'],
               [1, 1, 1, 1, 'yes'],
               [1, 0, 1, 2, 'yes'],
               [1, 0, 1, 2, 'yes'],
               [2, 0, 1, 2, 'yes'],
               [2, 0, 1, 1, 'yes'],
               [2, 1, 0, 1, 'yes'],
               [2, 1, 0, 2, 'yes'],
               [2, 0, 0, 0, 'no']]
    labels = ['年龄', '有工作', '有自己的房子', '信贷情况']
    return dataSet, labels

In [17]:
# 计算经验熵
def cal_empirical_entropy(data_vector):
    nums_data=len(data_vector)   #数据集样本数
    counts_by_labels={}      #用来保存每个label下的样本数
    entroy=0
    for vector in data_vector:
        if vector[-1] not in counts_by_labels:       #vector[-1]为label值
            counts_by_labels[vector[-1]]=0
        counts_by_labels[vector[-1]]+=1              #统计label出现的次数
    for key in counts_by_labels:
        p=float(counts_by_labels[key]/nums_data)          #计算每个标签出现的概率
        entroy-=p*math.log(p,2)                     #计算经验熵，公式5.7
    return entroy


In [18]:
# 划分数据集
# index_feature 特征的索引位置
# value 划分特征的取值
def split_datatset(data_vector,index_feature,value):
    split_set=[]
    for vector in data_vector:
        # 挑选vector[index_feature]==value的数据
        if vector[index_feature]==value:
            # 去掉第i列特征
            split_1=vector[:index_feature]
            split_1.extend(vector[index_feature+1:])
            split_set.append(split_1)
    return len(split_set), split_set

In [19]:
# 返回较大一方所对应位置的值和索引
def choose_max(fea_x1,max_x2,fea_index1,max_indx2):
    if fea_x1>max_x2:
        return fea_x1,fea_index1
    else:
        return max_x2,max_indx2

In [27]:
# 选择最优分类特征
def choose_bestfeture(data_vector,create_alg_para):
    nums_data=len(data_vector) # 数据数量
    nums_feature=len(data_vector[0])-1       #每个样本所包含的特征个数
    empirical_entropy=cal_empirical_entropy(data_vector)   #计算经验熵
    max_information_gain=0                     #表示最大信息增益
    max_information_gain_ratio=0                 #表示最大的信息增益比
    best_index_feature=0                         #表示最优特征的索引位置index
    for i in range(nums_feature): # 遍历所有特征
        features_i_set=[vector[i] for vector in data_vector]           #提取第i个特征中所包含的可能取值
        features_i_set=set(features_i_set)             #对特征值去重
        conditional_entroy=0                        #表示每个特征的经验条件熵，公式5.8
        ha_d_entroy=0                               #表示数据集D关于特征A的值的熵Ha(D)，公式5.10
        for fea in features_i_set:
            nums_di,di_set=split_datatset(data_vector,i,fea) # 得到划分的特征与相应数据
            p_di=nums_di/nums_data                 #计算|Di|/|D|，公式5.8
            ha_d_entroy-=p_di*math.log(p_di,2)     #计算数据集D关于特征A的值的熵Ha(D)，参考公式5.10
            entroy_di=cal_empirical_entropy(di_set)   #计算子类的经验熵，公式5.8中的H(Di)
        fea_information_gain=empirical_entropy-conditional_entroy    #计算每个特征对应的信息增益，公式5.9
        fea_information_gain_ratio = fea_information_gain/ha_d_entroy # 信息增益比

        # 通过create_alg_para参数选择是ID3还是C4.5算法。
        if create_alg_para == "ID3":
            max_information_gain,best_index_feature=choose_max(fea_information_gain,max_information_gain,i,best_index_feature)
        elif create_alg_para == "C45":
            max_information_gain_ratio,best_index_feature=choose_max(fea_information_gain_ratio,max_information_gain_ratio,i,best_index_feature)
        else:
            exit("create_alg_para should be 'ID3' or 'C45'.")

    return best_index_feature


In [28]:
# 返回类列表中出现次数最多的类标签
def max_class(label_list):
    count_label={}
    for label in label_list:
        if label not in count_label:
            count_label[label]=0
        count_label[label]+=1
    #     选择字典value最大的所对应的key值
    return max(count_label,key=count_label.get)

In [29]:

class Decision_tree(object):
    def __init__(self,data_vector,labels,create_alg_para='C45'):
        # 数据集
        self.data_vector=data_vector
        # 特征标签
        self.labels=labels
        # 生成决策树的方法：ID3或者C45
        self.create_alg_para=create_alg_para

    # 生成决策树，返回决策树tree，字典形式
    def tree_main(self):
        tree=self.create_decision_tree(self.data_vector,self.labels)
        return tree


    """
    递归函数，用于生成每一个子树,并返回。
    《统计学习方法》ID3或C4.5算法
    data_vector：每一个待分类数据集
    labels：待分类特征标签

    """
    def create_decision_tree(self,data_vector,labels):
        nums_label=[vector[-1] for vector in data_vector]
        # 如果数据集中所有实例属于同一个类，则停止划分。返回该类 标签。
        if len(set(nums_label))==1:
            return nums_label[0]
        # print("a",'\n',data_vector)
        # 如果特征集只有一类时，即已经遍历完了所有特征，则停止划分。返回出现次数最多的类标签
        if len(data_vector[0])==1:
            return max_class(nums_label)

        best_index_feature=choose_bestfeture(data_vector,self.create_alg_para)    #选择最优特征
        best_feature_label=labels[best_index_feature]         #最优特征的标签
        myTree = {best_feature_label: {}}                    #子决策树，key为最优特征的标签，value为子决策树
        del (labels[best_index_feature])                    #删除已经使用过的最优特征标签
        best_feature_value = [vector[best_index_feature] for vector in data_vector]
        best_feature_set = set(best_feature_value )
        # 根据最优特征标签的属性值划分新的子数据集，并递归生成子树
        for f_value in best_feature_set:
            nums_data_vector,data_vector_split=split_datatset(data_vector,best_index_feature,f_value)
            myTree[best_feature_label][f_value]=self.create_decision_tree(data_vector_split,labels)
        return  myTree

    def cart(self):
        # CART算法参考下一篇博客
        pass

if __name__ == '__main__':
    dataSet, labels = createDataSet()
    # best=choose_bestfeture(dataSet)
    # print(best)

    # create_alg_para should be 'ID3' or 'C45'
    tree=Decision_tree(dataSet,labels,create_alg_para="C45")
    print(tree.tree_main())

{'有工作': {0: {'有自己的房子': {0: 'no', 1: 'yes'}}, 1: 'yes'}}
